## Dog Breed and Cat Breed Classification Using MobileNetv2 (CNN)

### Importing all the libraries

In [ ]:
import numpy as np

import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.models import Sequential  
from keras.layers import Dropout, Flatten, Dense  
from keras import applications  
from keras.utils.np_utils import to_categorical
from keras.applications import inception_v3

import matplotlib.pyplot as plt  
import math

### Check for tensorflow version

In [ ]:
tf.__version__

### Perimeters

In [ ]:
img_width, img_height = 224, 224  

weights_path = 'inception_model_2.h5'  

epochs = 10
batch_size = 32

### InceptionV3 model

In [ ]:
model = inception_v3.InceptionV3(weights='imagenet', include_top=False, pooling='avg')

### Training Set Data Preprocessing

https://keras.io/api/preprocessing/

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

train_set = train_datagen.flow_from_directory(  
                                   'dataset/train/tg_set',  
                                   target_size=(img_width, img_height),  
                                   batch_size=batch_size,  
                                   class_mode=None,  
                                   shuffle=False)

nb_train_samples = len(train_set.filenames) 
num_classes = len(train_set.class_indices) 

### InceptionV3 Bottleneck Feature (Training Set)

In [ ]:
bf_train = model.predict(train_set, batch_size=batch_size, verbose=1)
   
np.save('bf_train_inceptionv3.npy', bf_train)

### Test Set Preprocessing Images

https://keras.io/api/preprocessing/

In [ ]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test/tt_set',
                                            target_size=(img_width, img_height),
                                            batch_size = batch_size,
                                            class_mode = None)

nb_test_samples = len(test_set.filenames)  

### InceptionV3 Bottleneck Feature (Test Set)

In [ ]:
bf_test = model.predict(train_set, batch_size=batch_size, verbose=1)    
   
np.save('bf_test_inceptionv3.npy', bf_test)

### Validation Data Preprocessing Images

In [ ]:
validation_datagen = ImageDataGenerator(rescale = 1./255)
validation_set = test_datagen.flow_from_directory('dataset/val/val_set',
                                            target_size=(img_width, img_height),
                                            batch_size = batch_size,
                                            class_mode = None)

nb_validation_samples = len(test_set.filenames)

### InceptionV3 Bottleneck Feature (Validation Set)

In [ ]:
bf_validation = model.predict(train_set, batch_size=batch_size, verbose=1)    
   
np.save('bf_valiadtion_inceptionv3.npy', bf_validation)

### To Categorical Train, Test, Valiadtion Set

In [ ]:
datagen_top = ImageDataGenerator(rescale = 1./255)
training_set_top = datagen_top.flow_from_directory(  
         'dataset/train/tg_set',  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode='categorical',  
         shuffle=False) 

nb_train_samples = len(training_set_top.filenames)  
num_classes = len(training_set_top.class_indices)  

test_set_top = datagen_top.flow_from_directory(  
         'dataset/test/tt_set',  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode=None,  
         shuffle=False)

nb_test_samples = len(test_set_top.filenames)

validation_set_top = datagen_top.flow_from_directory(  
         'dataset/val/val_set',  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode=None,  
         shuffle=False)

nb_validation_samples = len(validation_set_top.filenames)

### Uploading Bottleneck Features (Training Data)

In [ ]:
train_data = np.load('bf_train_inceptionv3.npy')  

train_labels = training_set_top.classes  
   
train_labels = to_categorical(train_labels, num_classes=num_classes)

### Uploading Bottlneck Feature (Test Data)

In [ ]:
test_data = np.load('bf_test_inceptionv3.npy')  

test_labels = test_set_top.classes  

test_labels = to_categorical(test_labels, num_classes=num_classes)

### Uploading Bottlneck Feature (Validation Data)

In [ ]:
validation_data = np.load('bf_valiadtion_inceptionv3.npy')  

validation_labels = validation_set_top.classes  

validation_labels = to_categorical(validation_labels, num_classes=num_classes)

### CNN and MobileNet model

In [ ]:
model = Sequential()  
model.add(Dense(256, activation='relu'))  
model.add(Dropout(0.5))  
model.add(Dense(num_classes, activation='softmax'))  
   
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])  
   
history = model.fit(train_data, 
                    train_labels, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_data=(validation_data, validation_labels))  
   
model.save_weights(weights_path)  
   
(eval_loss, eval_accuracy) = model.evaluate(  
     test_data, test_labels, batch_size=batch_size, verbose=1)

print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))  
print("[INFO] Loss: {}".format(eval_loss))

In [ ]:
import os

img_folder_path = "dataset/train/tg_set"
output_labels = os.listdir(img_folder_path)
print(class_labels)

In [ ]:
import coremltools as ct

your_model = ct.converters.keras.convert('inception_model_2.h5', input_names=['image'],output_names=['output'], class_labels=output_labels, image_input_names='image')

your_model.save('inception.mlmodel')